In [ ]:
# 런타임 연결 해제 및 삭제

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('lang=ko_KR')
options.headless = True

<ipython-input-3-8e2c29e82798>:10: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [4]:
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:

  driver.get('https://www.coffeebeankorea.com/store/store.asp')
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  temp = soup.select('#storeListUL > li')
  for i in temp:
    print(i.select('div.store_txt > p.name > span')[0].text)


# # 사용후 종료
# driver.quit()

<ipython-input-4-a3de951bfe0c>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:


삼성봉은사거리점33m
코엑스인터점262m
삼성오크우드점316m
삼성봉은사로점442m
도심공항타워점495m
트레이드타워점507m
도심공항점560m
파르나스몰점626m
삼성로점632m
삼성루첸타워점845m


In [26]:
from tqdm import tqdm
from IPython.display import clear_output
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:    
  totals = []
  for num in tqdm(range(100)):
    driver.get('https://www.coffeebeankorea.com/store/store.asp')
    driver.execute_script(f"storePop2({num})")  
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    tbody_list = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody > tr')  
    result = []
    
    clear_output(wait= True)
    print(f'크롤링 결과 : {tbody_list}', end = '\n')

    for row in tbody_list:    
      result.append(row.find('td').text) 
    totals.append(result)

100%|██████████| 100/100 [03:57<00:00,  2.38s/it]

크롤링 결과 : []


In [ ]:
import pandas as pd
new_lists = [row for row in totals if len(row) > 0]
df = pd.DataFrame( new_lists)
df.to_csv('coffebean.csv', encoding='utf-8')
df
# 제목달고... 마지막 두개 컬럼의 내용을 합쳐서 하나의 컬럼으로 정리.

In [ ]:
# 동적페이지에서 브라우져의 객체를 가져올때 또는 컨트롤 할때 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:  
  # 1. 사이트오픈
  driver.get('https://www.koreabaseball.com/TeamRank/TeamRank.aspx') 
  time.sleep(1) 
  total = []

  for year in tqdm(range(1982,2024)):
    combobox = driver.find_element(By.ID,'cphContents_cphContents_cphContents_ddlYear')  
    combobox_select = Select(combobox)
    combobox_select.select_by_value(f'{year}')
    time.sleep(1) 

    # 2. 오픈된 사이트를 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')  
    tr_list = soup.select('#cphContents_cphContents_cphContents_udpRecord > table > tbody >tr')
    result = []

    clear_output(wait= True)
    print(f'크롤링 결과 : {tr_list}', end = '\n')

    for tr in tr_list:
      result.append([td.text for td in tr.find_all('td')])    
    total.append(result)

In [ ]:
df = pd.DataFrame(total)
df.to_csv('./kbo.csv',encoding='utf-8')
df.head()

In [56]:
# kbo에서 2023년 현재 달력옆에있는 버튼 클릭 이벤트
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:  
  # 1. 사이트오픈
  driver.get('https://www.koreabaseball.com/TeamRank/TeamRank.aspx') 
  button_el = driver.find_element(By.ID,'cphContents_cphContents_cphContents_btnPreDate')  
  button_el.click()
  time.sleep(1)  # 시간주지말고... 동적으로 페이지 로딩후 동작하는 라이브러리나 이베트등 찾아볼것 
  button_el = driver.find_element(By.ID,'cphContents_cphContents_cphContents_btnPreDate')  
  button_el.click()
  time.sleep(1) 

  # 2. 오픈된 사이트를 크롤링
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')  
  tr_list = soup.select('#cphContents_cphContents_cphContents_udpRecord > table > tbody >tr')
  for tr in tr_list:
    print([td.text for td in tr.find_all('td')])  

<ipython-input-56-458d345efa92>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:


['1', 'SSG', '8', '7', '1', '0', '0.875', '0', '7승0무2패', '6승', '2-0-1', '5-0-1']
['2', 'LG', '10', '7', '3', '0', '0.700', '1', '7승0무3패', '1승', '3-0-0', '4-0-4']
['2', 'NC', '10', '7', '3', '0', '0.700', '1', '6승0무4패', '5승', '5-0-1', '2-0-3']
['4', '두산', '9', '6', '3', '0', '0.667', '1.5', '6승0무4패', '2승', '4-0-3', '2-0-1']
['5', 'KT', '7', '3', '4', '0', '0.429', '3.5', '4승0무4패', '3패', '1-0-1', '3-0-3']
['5', 'KIA', '7', '3', '4', '0', '0.429', '3.5', '3승0무5패', '1승', '2-0-4', '1-0-1']
['7', '롯데', '8', '3', '5', '0', '0.375', '4', '4승0무5패', '1패', '3-0-3', '1-0-2']
['8', '키움', '9', '3', '6', '0', '0.333', '4.5', '4승0무6패', '5패', '3-0-2', '1-0-4']
['9', '삼성', '9', '2', '7', '0', '0.222', '5.5', '3승0무7패', '6패', '3-0-4', '0-0-3']
['9', '한화', '9', '2', '7', '0', '0.222', '5.5', '3승0무7패', '1패', '0-0-3', '3-0-4']


In [77]:
from selenium.webdriver.common import keys
# 검색
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:  
  # 1. 사이트오픈
  driver.get('https://www.koreabaseball.com/Default.aspx')   
  driver.find_element(By.ID,'popupVideo').click()
  time.sleep(1)
  search_button = driver.find_element(By.ID,'txtTopPlayerWord')  
  search_button.send_keys('추신수')
  search_button.send_keys(keys.Keys.RETURN)  # 엔터
  # search_button.submit()  #검색버튼 클릭  
  time.sleep(1)

  # 2. 오픈된 사이트를 크롤링
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')    
  tr_list = soup.select('#contents > div.sub-content > div.player_records > div:nth-child(6) > table > tbody > tr')
  print(tr_list)
  for tr in tr_list:
    print([td.text for td in tr.find_all('td')])  
  

<ipython-input-77-32a1eea70220>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:


[<tr>
<td>04.13</td>
<td>삼성</td>
<td>0.500</td>
<td>2</td>
<td>2</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>, <tr>
<td>04.12</td>
<td>삼성</td>
<td>0.000</td>
<td>5</td>
<td>5</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>2</td>
<td>0</td>
</tr>, <tr>
<td>04.11</td>
<td>삼성</td>
<td>0.500</td>
<td>5</td>
<td>4</td>
<td>2</td>
<td>2</td>
<td>0</td>
<td>0</td>
<td>1</td>
<td>1</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>0</td>
</tr>, <tr>
<td>04.09</td>
<td>한화</td>
<td>0.000</td>
<td>4</td>
<td>4</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>1</td>
<td>0</td>
</tr>, <tr>
<td>04.08</td>
<td>한화</td>
<td>0.667</td>
<td>6</td>
<td>3</td>
<td>1</td>
<td>2</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>3</td>

In [60]:
# # 사용후 종료
driver.quit()